<a href="https://colab.research.google.com/github/anandv01/ELL409-Fall-2019-Machine-Intelligence-and-Learning/blob/master/UPSC_CSE_QCAB_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script generates PDF file of Question cum Answer Booklet (QCAB) for the UPSC CSE Mains Examination from the given list of questions.

Important Points:


*   Download the font and add its path in the script.

  Link to download PT Serrif Font: https://fonts.google.com/specimen/PT+Serif

  `You can use any font of your choice including Hindi`


*   Provide list of questions in the CSV file as an input.

  `Save it as CSV file with utf-8 encoding.`

  Sample Dummy Data in Required Format:


  |Page Count|Footer Stamp|Question Number|Question Text|Question Stamp|
  |---|---|---|---|---|
  |1|XXXX-2013-GS1|Q\. 1|Bring out the causes for more frequent landslides in the Himalayas than in Western Ghats|5  Marks,  2013|
  |1|XXXX-2013-GS1|Q\. 2|Major hot deserts in northern hemisphere are located between 20-30 degree north and on the western side of the continents\. Why?|5  Marks,  2013|
  |1|XXXX-2013-GS1|Q\. 3|There is no formation of deltas by rivers of the Western Ghat\. Why?|5  Marks,  2013|
  |1|XXXX-2013-GS1|Q\. 4|The recent cyclone on the east coast of India was called “Phailin”\. How are the tropical cyclones named across the world?|5  Marks,  2013|
  |1|XXXX-2013-GS1|Q\. 5|What do you understand by the theory of continental drift? Discuss the prominent evidences in its support\.|10  Marks,  2013|
  |1|XXXX-2013-GS1|Q\. 6|Why are the world’s fold mountain systems located along the margins of continents? Bring out the association between the global distribution of Fold Mountains and the earthquakes and volcanoes\.|10  Marks,  2013|
  |2|XXXX-2014-GS1|Q\. 8|Tropical cyclones are largely confined to South China Sea, Bay of Bengal and Gulf of Mexico\. Why?|10  Marks,  2014|
  |2|XXXX-2014-GS1|Q\. 9|Explain the formation of thousands of islands in Indonesian and Philippines archipelagos\.|10  Marks,  2014|
  |2|XXXX-2016-GS1|Q\. 10|“The Himalayas are highly prone to landslides\.” Discuss the causes and suggest suitable measures of mitigation\.|12\.5  Marks,  2016|
  |2|XXXX-2017-GS1|Q\. 11|How does the Juno Mission of NASA help to understand the origin and evolution of the Earth?|12\.5  Marks,  2017|
  |2|XXXX-2018-GS1|Q\. 12|Define mantle plume and explain its role in plate tectonics\.|12\.5  Marks,  2018|
  |3|XXXX-2018-GS1|Q\. 13|Why is Indian Regional Navigational Satellite System \(IRNSS\) needed? How does it help in navigation?|15  Marks,  2018|
  |3|XXXX-2020-GS1|Q\. 14|The process of desertification does not have climatic boundaries\. Justify with examples\.|15  Marks,  2020|
  |3|XXXX-2020-GS1|Q\. 15|Discuss the geophysical characteristics of Circum-Pacific Zone\.|15  Marks,  2020|
  |3|XXXX-2021-GS1|Q\. 16|Mention the global occurrence of volcanic eruptions in 2021 and their impact on regional environment\.|15  Marks,  2021|
  |3|XXXX-2021-GS1|Q\. 17|Why is India considered as a subcontinent? Elaborate your answer|15  Marks,  2021|
  |4|XXXX-2021-GS1|Q\. 18|Differentiate the causes of landslides in the Himalayan region and Western Ghats\.|20  Marks,  2021|
  |4|XXXX-2021-GS1|Q\. 19|Briefly mention the alignment of major mountain ranges of the world and explain their impact on local weather conditions with examples\.|20  Marks,  2021|
  |4|XXXX-2022-GS1|Q\. 20|Describe the characteristics and types of primary rocks\.|20  Marks,  2022|

In [ ]:
!pip install fpdf2

In [ ]:
import warnings
warnings.simplefilter('default', DeprecationWarning)

from fpdf import FPDF
from base64 import b64encode
from IPython.display import display, HTML
import csv

In [ ]:
class PDF(FPDF):
    def update_setting(self):

        self.set_margin(0)

        global pageHeight
        global pageWidth
        global qStampWidth

        global leftMargin_x
        global topMargin_y
        global rightMargin_x
        global bottomMargin_y

        pageHeight = self.eph
        pageWidth = self.epw
        qStampWidth = 60

        #set inner margin
        leftMargin_x = 25
        topMargin_y = 15
        rightMargin_x = self.epw-25
        bottomMargin_y = self.eph-15

    def add_leftLine(self):
        self.set_line_width(0.3)
        self.set_draw_color(r=0, g=0, b=0)
        self.line(x1=leftMargin_x, y1=topMargin_y, x2=leftMargin_x, y2= bottomMargin_y)

    def add_rightLine(self):
        self.set_line_width(0.3)
        self.set_draw_color(r=0, g=0, b=0)
        self.line(x1=rightMargin_x, y1=topMargin_y, x2=rightMargin_x, y2= bottomMargin_y)


    def add_footer(self, textContent):
        self.set_xy(leftMargin_x-10, bottomMargin_y+3)
        self.set_font("Times", "I", 12)
        self.cell(0, None, textContent, align="L")

    def add_leftText(self, textContent):
        self.set_xy(leftMargin_x-20, topMargin_y)
        self.set_font("PTSerif", size=12)
        self.multi_cell(15, None, textContent, align="C")

    def add_rightText(self, textContent):
        self.set_xy(rightMargin_x+2, topMargin_y)
        self.set_font("Times", size=9)
        self.multi_cell(0, None, textContent, align="L")

    def add_questionText(self, textContent, miscContent = None):
        self.set_font("PTSerif", size=12)

        self.set_xy(leftMargin_x+2, topMargin_y)
        localbox_width = rightMargin_x-leftMargin_x-4
        self.multi_cell(localbox_width, None, textContent, align="J")

        #add marks stamp in next line
        if (miscContent != None):
          marksStamp = "["+ str(miscContent) +"]"
          self.set_xy(rightMargin_x-qStampWidth, self.y)
          self.multi_cell(qStampWidth-2, None, marksStamp, align="R")



    def add_layout(self, footerMark):
        self.add_leftLine()
        self.add_rightLine()
        self.add_footer(footerMark)
        self.add_rightText("Candidates must not write on this margin")

    def add_questionContent(self, qNumber, qText, qStamp = None):
          self.add_leftText(qNumber)
          self.add_questionText(qText, qStamp)

    def add_question(self, pageCount, footerMark, qNumber, qText, qStamp = None):
        for i in range(pageCount):

            self.add_page()
            self.add_layout(footerMark)

            if (i==0):
                self.add_questionContent(qNumber, qText, qStamp)


In [ ]:
# Generate and Download the PDF:

# Reference to font file and CSV File
fontPath = '/content/PTSerif-Regular.ttf' # Replace it with location of font file path in your local environment
questionCSVPath = '/content/QCABDummyData.csv' # Replace it with location of question csv file path in your local environment

pdf = PDF()
pdf.add_font(family ='PTSerif', fname = fontPath)
pdf.update_setting()

with open(questionCSVPath, 'r') as f:
    reader = csv.reader(f)
    for i, line in enumerate(reader):
      if (i>0):
        a0, a1, a2, a3, a4 = line
        pageCount = int(a0)
        footerMark = a1
        qNumber = a2
        qText  = a3
        qStamp = a4

        pdf.add_question(pageCount, footerMark, qNumber, qText, qStamp)


pdf_bytes = pdf.output()
base64_pdf = b64encode(pdf_bytes).decode("utf-8")
display(HTML(f'<a download="upsc-qcab.pdf" href="data:application/pdf;base64,{base64_pdf}">Click to download PDF</a>'))

Connect with me at https://www.linkedin.com/in/anand-kumar-verma/ 😀